# LangChain Tools Tutorial

This notebook demonstrates how to work with tools in LangChain, including built-in tools and creating custom tools. We'll cover:

1. **Built-in Tools**: DuckDuckGo Search and Shell tools
2. **Custom Tools**: Three different methods to create custom tools
3. **Toolkits**: Organizing multiple tools together

Let's start by installing the required packages.


## Package Installation

First, we need to install the required LangChain packages and dependencies. This includes:
- `langchain`: Core LangChain framework
- `langchain-core`: Core abstractions and interfaces
- `langchain-community`: Community-contributed tools and integrations
- `pydantic`: Data validation and settings management
- `duckduckgo-search`: Search functionality
- `langchain_experimental`: Experimental features and tools


In [1]:
!pip install -q langchain==0.3.27 langchain-core==0.3.79 langchain-community==0.3.31 pydantic==2.11.10 duckduckgo-search==8.1.1 langchain_experimental==0.3.4 ddgs==9.6.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
!pip show langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental ddgs

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community
---
Name: langchain-core
Version: 0.3.79
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity, typing-extensions
Required-by: langchain, langchain-community, langchain-experimental, langchain-text-splitters
---
Name: langchain-community
Version: 0.3.31
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain, langchain-core, langsmith,

## Built-in Tool - DuckDuckGo Search

## DuckDuckGo Search Tool

The DuckDuckGo Search tool is a built-in tool in LangChain that allows you to perform web searches. This is particularly useful for getting current information or answering questions about recent events.

Let's create an instance of the search tool and test it with a search query.


In [3]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('top news in india today')

print(results)

Get the latest updates and breaking news from India , including politics, elections, government, business, technology, and Bollywood. This India Today special report, anchored by Gaurav Sawant, decodes Prime Minister Narendra Modi's Deepavali address from the indigenous aircraft carrier, INS Vikrant. The Prime Minister's statement, referencing 'Operation Sindoor' and Pakistan, sent a strategic message to India's adversaries. Top military experts, including Vice Admiral Shekhar Sinha and Mrs.3 Mrs.2 Syed Ata Hasnain, analyze ... News Today Live Updates: Get the latest news updates from India and all around the world. Stay updated with breaking news today . Top News Headlines of October 20, 2025 | News Today Live Updates, 20 October: As Delhi gears up to celebrate Diwali today , the city is already wrapped in a thick layer of smog, making it increasingly difficult to breathe. Air quality across the national capital has ... Today's top news : Delhi's air quality worsens, Trump warns India

## Tool Properties

Every tool in LangChain has several important properties that describe its functionality:

- **name**: The unique identifier for the tool
- **description**: A human-readable description of what the tool does
- **args**: A schema describing the input parameters the tool expects

Let's examine these properties for our search tool.


In [4]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## Built-in Tool - Shell Tool

## Shell Tool

The Shell tool allows you to execute shell commands directly from your Python code. This can be useful for file operations, system administration tasks, or running external programs.

**⚠️ Warning**: The shell tool has no safeguards by default and should be used with caution, especially in production environments.


In [5]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('ls')

print(results)

Executing command:
 ls
sample_data



/usr/local/lib/python3.12/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


## Custom Tools

## Creating Custom Tools

While LangChain provides many built-in tools, you'll often need to create custom tools for your specific use cases. LangChain offers three main approaches to create custom tools:

1. **Using the `@tool` decorator** (simplest approach)
2. **Using `StructuredTool.from_function()`** (more control over schema)
3. **Inheriting from `BaseTool`** (most flexible approach)

Let's explore each method using a simple multiplication function as an example.


### Method 1: Using the `@tool` Decorator

This is the simplest way to create a custom tool. We'll start by creating a basic function and then gradually add the necessary components to make it a LangChain tool.

**Step 1**: Create a basic function without any LangChain-specific features.


**Step 2**: Add type hints to make the function more robust and help LangChain understand the expected input and output types.


**Step 3**: Apply the `@tool` decorator to convert the function into a LangChain tool. The decorator automatically extracts type information and creates the necessary tool metadata.


In [6]:
from langchain_core.tools import tool

In [7]:
# Step 1 - create a function

def multiply(a, b):
    """Multiply two numbers"""
    return a*b

In [8]:
# Step 2 - add type hints

def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [9]:
# Step 3 - add tool decorator

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

Now let's test our custom tool by invoking it with some sample values.


In [10]:
result = multiply.invoke({"a":3, "b":5})

In [11]:
print(result)

15


Let's examine the tool's properties to see how LangChain has automatically generated the metadata from our function.


In [12]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


We can also access the JSON schema of the tool's arguments, which is useful for integration with other systems or for documentation purposes.


In [13]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## Method 2 - Using StructuredTool

### Method 2: Using StructuredTool

The `StructuredTool` approach gives you more control over the tool's schema and metadata. This method is useful when you need custom field descriptions or more complex validation rules.

First, we need to import the necessary classes and define a Pydantic model for our input schema.


In [14]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

Define a Pydantic model that describes the input schema for our tool. This allows us to provide custom descriptions and validation rules for each parameter.


In [15]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

Create the function that will be wrapped by the tool. This is the actual implementation of our multiplication logic.


In [16]:
def multiply_func(a: int, b: int) -> int:
    return a * b

Create the tool using `StructuredTool.from_function()`. This method allows us to specify the function, name, description, and argument schema explicitly.


In [17]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

Test the tool and examine its properties to see how our custom schema descriptions are reflected in the tool metadata.


In [18]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


### Method 3: Using BaseTool Class

The most flexible approach is to inherit from the `BaseTool` class. This gives you complete control over the tool's behavior and is useful for complex tools that need custom logic beyond just function execution.

First, let's import the necessary classes and redefine our input schema.


In [19]:
from langchain.tools import BaseTool
from typing import Type

Define the input schema using Pydantic. This is the same as in Method 2, but we'll use it in a different way.


In [20]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

Create a custom tool class by inheriting from `BaseTool`. This approach allows you to:
- Define tool metadata as class attributes
- Implement custom logic in the `_run` method
- Add additional methods for more complex behavior
- Override other methods like `_arun` for async support


Instantiate our custom tool class to create a tool instance.


In [21]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

Test the custom tool and examine its properties to verify everything works correctly.


In [22]:
multiply_tool = MultiplyTool()

In [23]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Creating Toolkits

A toolkit is a collection of related tools that can be used together. This is useful for organizing multiple tools that serve a common purpose, such as a set of mathematical operations or data processing tools.

Let's create a simple math toolkit that includes both addition and multiplication tools.


First, let's create our custom tools using the `@tool` decorator. We'll create both an addition and multiplication tool.


In [24]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


Create a toolkit class that organizes our tools. The `get_tools()` method returns a list of all tools in the toolkit, making it easy to access them as a group.


In [25]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]


Test our toolkit by creating an instance and examining all the tools it contains. This demonstrates how toolkits make it easy to manage and access multiple related tools.


In [26]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers
